# Tutorial 3: Loops and Subroutines in Julia

We've come to a point where weird and interesting things in Julia start to appear. Since Julia incorporates features of many languages, some of which are not very popular, a foray into serious Julia programming might introduce you to some "syntactic peculiarities" or behaviors that sometimes knock you off your feet if your programming life has recently been heavily dependent on just a few languages. With loops, for instance, there are some conventions that are similar to Matlab, while recursion looks familiar enough to those used to functional programming. Subroutines, or *functions*, start to show slivers of Julia's central programming paradigm, *i.e.*, **multiple dispatch**. And argument passing follows a convention called "pass by sharing", which might sound foreign (or not) to many programmers (although it is heavily used in Python and JavaScript albeit under different nomenclatures).

## 1. Loops

Let's start with something less surprising first: loops. Loops in Julia are pretty similar to those in Python and Matlab. We've got our good old friend `while`(-do) loop:

In [1]:
i = 3
while i > 0
    println("current i = ", i)
    i -= 1
end 

current i = 3
current i = 2
current i = 1


Julia does not have dedicated keywords for do-while loops, but you can always get around by `while true` and having a `break` condition near the end of the `while` block:

In [2]:
i = 3
while true
    println("current i = ", i)
    i -= 1
    i > 0 || break # hey, I'm starting to get addicted to this syntactic sugar,
                   # which was introduced in Tutorial 2. You can replace this 
                   # by the good old-fashioned `if-break` combo.
end 

current i = 3
current i = 2
current i = 1


The syntax for `for` loops is very similar to that in Matlab (with a bit of Python): you construct a range, whose both ends are **inclusive**, by stating the starting point (conventionally `1`) and the stopping point. If you want to loop 3 times, the range is `1:3`. The syntax then is

In [3]:
for i in 1:3
    println(i)
end

1
2
3


`for`-each, *e.g* when iterating items in an array, follows the same syntax:

In [4]:
for i in [1 2 3]
    println(i)
end

1
2
3


If you *really* miss Matlab, the following syntax for `for` and `for`-each is exactly the same as that in Matlab:

In [5]:
for i = 1:3
    println(i)
end

1
2
3


In [6]:
for i = [1 2 3]
    println(i)
end

1
2
3


You know what also resembles Matlab? The syntax for ranges with increments other than 1. The formula is `<start>:<step>:<stop>` and used like this:

In [7]:
for i = 1:2:6
    println(i)
end

1
3
5


You can also iterate backwards:

In [8]:
for i = 6:-2:1
    println(i)
end

6
4
2


## 2. Function

### 2.1. A glimpse of ✨✨✨<span style="color:violet">***MULTIPLE DISPATCH***</span>✨✨✨

Things start to look either wacky or exciting when it comes to *functions*. Look at this:

In [9]:
+(1, 2)

3

`+` is the alias of a function provided by off-the-shelf Julia, which is defined only for a few combinations of types such as subtypes of `Real`, *e.g.*, `Int` and `Float` and `Bool` and stuff like that, or `Real` and `Complex`, or `Complex` and `Complex`. What if you create a whole new type and want to `+` objects of that type together?

Let's say we have a `Point` type, which is defined as follows:

In [10]:
mutable struct Point
    x::Real # x-coordinate
    y::Real # y-coordinate
end

Now we want to seamlessly add 2 points together using the `+` operator in the same fashion as adding numbers. To do that, you must *overwrite* the `+` operator so that it works with `Point` inputs. The following code does exactly that:

In [11]:
import Base.+ # import this buit-in function before modifying
function +(a::Point, b::Point)
    return Point(a.x + b.x, a.y + b.y)
end

a = Point(1, 2.5)
b = Point(1.5, 3)
println(a + b)

Point(2.5, 5.5)


Wow, that's so cool, right? Having the ability to modify built-in functions like `+` opens up a host of possibilities (and risks) that are only limited by your imagination. Let's say your imagination entails that a `Point` can also be added with a `Real` number, whereby each coordinate is added with that number. You just, once again, define another `+` function for that behavior: 

In [12]:
function +(a::Point, s::Real)
    Point(a.x + s, a.y + s)
end

a = Point(1, 2.5)
s = 1.5
println(a + s)

Point(2.5, 4.0)


And you can still `+` 2 `Point`s together after `+` has been modified (or, to be more accurate, extended):

In [13]:
c = Point(3, 4)
d = Point(2.5, 6)
println(c + d)

Point(5.5, 10)


This ability of defining multiple functions with the same name (or, you can say, endowing a function with many different behaviors) is called **multiple dispatch**. And it's a very big deal in Julia, since the language was basically built around a paradigm that is called--you guessed it--**multiple dispatch**. 

I'm gonna capitalize, italicize, bold that term, and then color it in violet, and then decorate it with some sparkles, to give you a sense of how important it is:

✨✨✨<span style="color:violet">***MULTIPLE DISPATCH***</span>✨✨✨

Why is it so important? Because it implements this whole idea of Julia being a language for math and science and data: instead of modelling objects and their individual characteristics and abilities (like in OOP languages), Julia programmers model *relationships*. So instead of modelling, for instance, all the nuts and bolts and behaviors of a `Point` object (including the various functions by which it is `+`ed with other objects), we model how `Point`s are related to other types by defining relationships such as that represented by `+`. Functions are now first-class citizens, they become **polymorphic**, and each of their variants, which is called a **method**, map a set of **types** to a return value. This also partially explains the complicated type system that we briefly looked at in Tutorial 1: there are many types of numbers with all sorts of hierarchies because they interact in complicated ways in terms of both math and data structure.

We will get back to this idea in the next tutorial. Right now, we can just run away with the impression that multiple dispatch == one function having multiple methods, and that it's a big deal in terms of language design.

A small note: although you've mostly seen functions whose argument types are defined, you can always build those that accept arguments whose typed are not defined until runtime, such as a function that accepts a pair of any types as long as their exists a `+` method for them.

### 2.2. Function order

The order of function definitions is not important, but function calls outside of function scopes should be placed after the functions have been defined.

In [ ]:
function area(a)
    return circscribed_circle(a) - equi_triangle(a)
end

function equi_triangle(a)
    return sqrt(3) / 4 * a^2
end

function circscribed_circle(a)
    a^2 * (pi/3) # `return` keyword is optional, so it can be done without
end

area(3)

So this is not legal:

In [18]:
area(3)

function area(a)
    return circscribed_circle(a) - equi_triangle(a)
end

function equi_triangle(a)
    return sqrt(3) / 4 * a^2
end

function circscribed_circle(a)
    a^2 * (pi/3) # `return` keyword is optional, so it can be done without
end

UndefVarError: UndefVarError: area not defined

### 2.3. Pass-by-sharing

In Julia, arguments are passed into methods by sharing. This may sound unfamiliar, but that's just a way of saying that it is similar to what happens in Python and JavaScript. Consider this function:

In [14]:
function printpoint(point::Point)
    println("(x=", point.x, ", y=", point.y, ")")
end

a = Point(1, 2.5)
printpoint(a)

(x=1, y=2.5)


When the name/variable `a` is passed in to `printpoint`, the address which `a` points at is now also pointed by the name/variable `point` that exists only within the scope of the function. "Sharing" in "pass-by-sharing" now means that at some point during execution, there are at least 2 names that share the job of pointing at a single value.

### 2.4. Keyword arguments

Imagine that upon defining a function, you realize that it has too many arguments. While multiple dispatch, *i.e.* defining different methods of that function so that each only work with a subset of arguments, might sometimes help, usually you just cannot help but keep them all. And that becomes a nightmare whenever you need to use that function, because you will forget the order of the arguments.

That's when keyword arguments are absolutely necessary. A function can accept regular arguments and keyword arguments at the same time using this syntax:

```
function f(args; kwarg1, kwarg2)
```

When you define that function using only keyword arguments,

```
function f(; kwarg1, kwarg2)
```

Here's an example where a function accepts both kinds of arguments:

In [15]:
function add_coords(a::Point; i, j)
    x_b = a.x + i
    y_b = a.y + j
    return Point(x_b, y_b)
end

a = Point(1, 2.5)
b = add_coords(a; j=1.5, i=0) # note that the order of the keyword args is 
                              # the reverse of that in the function definition,
                              # but it's totally ok.
                              # Regular arguments have to be put first, tho,
                              # and in the same order as that in the func def. 
println("after adding 0 to x, 1.5 to y: ", b)

after adding 0 to x, 1.5 to y: Point(1, 4.0)


### 2.5. Multiple returns

Like Python, Julia can return multiple things at once in the form of a tuple. Tuple unpacking is also similar.

In [22]:
function extractcoords(a::Point)
    a.x, a.y
end

a = Point(1, 2.5)
println(extractcoords(a))
println("return type: ", typeof(extractcoords(a)))

x, y = extractcoords(a) # tuple unpacking
println("tuple unpacked: x = ", x, ", y = ", y)

(1, 2.5)
return type: Tuple{Int64, Float64}
tuple unpacked: x = 1, y = 2.5


### 2.6. Methods that modify arguments

Julia programmers are pretty serious about side effects, *i.e.*, the result of a function relying on or changing things outside its scope. That's understandable, because one design principle of Julia is to utilize parallel computing whenever possible, to which side effects are vermin.

Although Julia does not have hard-and-fast mechanisms to absolutely prevent side effects, programmers follow a naming convention whereby functions that *might* cause side effects have a lovely `!` at the end of the name. For instance, 

In [16]:
function scalarmul!(a::Point, s::Real) # multiply the coordinates with a scalar
                                       # in-place, i.e., with side effects
    a.x = a.x * s
    a.y = a.y * s
end

a = Point(1, 2.5)
scalarmul!(a, 2)
println(a)

Point(2, 5.0)


### 2.7. Higher-order functions

Did I tell you that you can do functional programming with Julia? One example of features for functional programming is higher-order functions, which are used extensively in Julia codebases. What that means is that a function can be passed as an argument or can be returned by another function, and that other function is higher-order. 

This feature entails the necessity of anonymous functions defined by lamdbda expressions with the syntax

```
(<arguments>) -> <statement>
```

Remember ternary statements? Lambdas and ternaries are like peanut butter and jelly. The example below shows how a ternary statement is used in a lambda expression, beside being passed into a higher-order function.

In [17]:
function broadcast(f::Function, a::Point, args...; kwargs...)
                                       # `args...` is for an unspecified list of
                                       # positional arguments, while `kwargs...`
                                       # is for keyword arguments.
    x = f(a.x, args...; kwargs...)
    y = f(a.y, args...; kwargs...)
    Point(x, y)
end

function cutoff(x; low, high)
    low < x < high ? x : 0
end

highpass = (x, threshold) -> x > threshold ? x : 0 # haha, see? it looks so cool

a = Point(1, 2.5)
a_double = broadcast(*, a, 2)
a_cutoff = broadcast(cutoff, a; low=0, high=1.5)
a_highpass = broadcast(highpass, a, 1.5)
println("coordinates of a after being doubled: ", a_double)
println("coordinates of a cut off at low=0, high=1.5: ", a_cutoff)
println("coordinates of a highpassed at 1.5: ", a_highpass)

coordinates of a after being doubled: Point(2, 5.0)
coordinates of a cut off at low=0, high=1.5: Point(1, 0)
coordinates of a highpassed at 1.5: Point(0, 2.5)


### 2.8. Recursive functions

Since functional programming can be done in Julia, recursive functions should exist and be used extensively. In fact, we came across a recursive function in the last tutorial, which was used to demonstrate shorthand `if`-`else`s. I'll just copy-paste it here.

In [25]:
function fib(n::Int)
    n >= 0 || error("Input must not be negative.")
    n <= 1 && return n
    fib(n-1) + fib(n-2) # does not need the `return` keyword 
end

fib(3)

2

## Conclusion

This tutorial is a whirlwind overview of loops and functions in Julia, which boast many beautiful features that the Julia community is very proud of. One of which is multiple dispatch, which is core to typical Julia workflows. Basically, multiple dispatch allows defining many methods for the same function, each of which maps a set of argument types to a set of returns. Think of polymorphism but for functions instead of objects!

Other cool things are that `for` loops are similar to those in Matlab (which might not be cool for some people), pass-by-sharing, keyword arguments, multiple returns, a convention to signal side effects, and features for functional programming like higher-order functions, lambda expressions, and recursion.